In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-12 22:41:41--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  1.81MB/s    in 0.6s    

2024-03-12 22:41:42 (1.81 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text))) # 알파벳, 숫자, 특수문자 등등
vocab_size = len(chars)
print(''.join(chars))   
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [8]:
# 입력값을 토큰화 
# 일반적으로 사람들은 sub word encoding을 사용하지만, 단순하게 유지하기 위해 우리가 사용하는 것은 문자 레벨 수준의 토크나이저. 

stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)} 

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# 두 개의 정수를 임의의 문자열로 변환하고 다시 변환하는 것과 같음

print(encode('hii there'))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [9]:
# encode된 텍스트를 파이토치 텐서로 변환합니다.
# 아래의 정수 뭉치들은 위에서 input.txt의 [:1000]과 동일한 텍스트를 나타냅니다.

import torch 

data = torch.tensor(encode(text), dtype=torch.long)

print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [10]:
# 훈련데이터와 검증 데이터를 분할합시다.

n = int(0.9*len(data)) # 90%를 훈련데이터로 사용
train_data = data[:n]
val_data = data[n:] 

In [11]:
# transformer를 훈련할 때 모든 텍스트를 넣으면 비용이 많이 들기때문에, chunk 단위로 덩어리를 나눠서 사용합니다.

block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) 이 값들을 우리가 Transformer에 연결할 때, 이것이 말하는 것은 실제로 동시에 훈련하여 현재의 모든 위치에서 예측을 할 것이라는 것입니다.
9개의 문자 덩어리에는 실제로 8개의 개별 예시가 포함되어 있습니다. 

위의 예시에서 47의 맥락에서 보았을 때 18의 다음에 올 가능성이 높고, 18 47 56의 맥락에서 다음으로 57이 올 수 있다는 것을 알 수 있습니다.  

In [12]:
# x는 Transformer에 대한 입력이며 첫 번째 블록 크기 문자가 될 것입니다. 
# y는 다음 블록 크기가 됩니다. 문자이므로 1만큼 오프셋됩니다. 
# 하나의 컨텍스트부터 블록 크기의 컨텍스트까지를 사용하여 여기에 있는 8개의 예제 모두에 대해 훈련합니다.
# Transformer는 하나의 컨텍스트를 사용하여 다음 문자를 예측하고, 그 예측을 사용하여 다음 문자를 예측합니다. 이렇게 하면 블록 크기의 모든 것을 예측할 수 있고 블록 크기 이후에는 자르기를 시작합니다.
# 왜냐하면 Transformer는 다음 문자를 예측할 때 블록 크기 이상의 입력을 받지 않기 때문입니다. 

x = train_data[:block_size]
y = train_data[1:block_size+1]  

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58
